# TODO 3: Adding new line connection
## 1. Find the exact cordinate of the targeted line to connects

In [72]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import yaml
import math
from shapely import LineString, Point, Polygon, distance


In [73]:
REGIONS_CONFIG = "regions_definition_config.yaml"

def is_nan(value):
    return math.isnan(float(value))

def read_osm_config(*args):
    """
    Read values from the regions config file based on provided key arguments.

    Parameters
    ----------
    *args : str
        One or more key arguments corresponding to the values to retrieve
        from the config file. Typical arguments include "world_iso",
        "continent_regions", "iso_to_geofk_dict", and "osm_clean_columns".

    Returns
    -------
    tuple or str or dict
        If a single key is provided, returns the corresponding value from the
        regions config file. If multiple keys are provided, returns a tuple
        containing values corresponding to the provided keys.

    Examples
    --------
    >>> values = read_osm_config("key1", "key2")
    >>> print(values)
    ('value1', 'value2')

    >>> world_iso = read_osm_config("world_iso")
    >>> print(world_iso)
    {"Africa": {"DZ": "algeria", ...}, ...}
    """
    if "__file__" in globals():
        base_folder = os.path.dirname(__file__)
        if not os.path.exists(os.path.join(base_folder, "configs")):
            base_folder = os.path.dirname(base_folder)
    else:
        base_folder = os.getcwd()
    osm_config_path = os.path.join(base_folder, "configs", REGIONS_CONFIG)
    with open(osm_config_path, "r") as f:
        osm_config = yaml.safe_load(f)
    if len(args) == 0:
        return osm_config
    elif len(args) == 1:
        return osm_config[args[0]]
    else:
        return tuple([osm_config[a] for a in args])

def read_geojson(fn, cols=[], dtype=None, crs="EPSG:4326"):
    """
    Function to read a geojson file fn. When the file is empty, then an empty
    GeoDataFrame is returned having columns cols, the specified crs and the
    columns specified by the dtype dictionary it not none.

    Parameters:
    ------------
    fn : str
        Path to the file to read
    cols : list
        List of columns of the GeoDataFrame
    dtype : dict
        Dictionary of the type of the object by column
    crs : str
        CRS of the GeoDataFrame
    """
    # if the file is non-zero, read the geodataframe and return it
    if os.path.getsize(fn) > 0:
        return gpd.read_file(fn)
    else:
        # else return an empty GeoDataFrame
        df = gpd.GeoDataFrame(columns=cols, geometry=[], crs=crs)
        if isinstance(dtype, dict):
            for k, v in dtype.items():
                df[k] = df[k].astype(v)
        return df

def save_to_geojson(df, fn):
    if os.path.exists(fn):
        os.unlink(fn)  # remove file if it exists

    # save file if the (Geo)DataFrame is non-empty
    if df.empty:
        # create empty file to avoid issues with snakemake
        with open(fn, "w") as fp:
            pass
    else:
        # save file
        df.to_file(fn, driver="GeoJSON")

In [74]:
RDIR = "SEA_IRENA_2050/"
fn_lines = "../../../pypsa-earth/resources/" + RDIR + "osm/clean/all_clean_lines_original.geojson"
fn_buses = "../../../pypsa-earth/resources/" + RDIR + "osm/clean/all_clean_substations_original.geojson"

osm_clean_columns = read_osm_config("osm_clean_columns")

lines = read_geojson(
    fn_lines,
    osm_clean_columns["line"].keys(),
    dtype=osm_clean_columns["line"],
)

buses = read_geojson(
    fn_buses,
    osm_clean_columns["substation"].keys(),
    dtype=osm_clean_columns["substation"],
)

In [75]:
def create_custom_network(df, line_sign, bus0_sign, bus1_sign):
    d_lines = { 'line_id': [f"{line_sign}000{i}-0" for i in df.index],
                'tag_type': ["line" for i in df.index],
                'voltage': df.Voltage * 1000,
                'tag_frequency' : [50 for i in df.index],
                'circuits' : [0 for i in df.index],  #df.Circuit,
                'bus0' : [Point(xy) for xy in zip(df.x1,df.y1)],
                'bus1' : [Point(xy) for xy in zip(df.x2,df.y2)],
                'length' : [None for i in df.index],
                'underground' : [False for i in df.index],
                'under_construction' : [False for i in df.index],
                'dc' : df.DC,
                'country' : df.Country,
    }
    df_lines = pd.DataFrame(data = d_lines)
    df_lines.loc[df_lines['country'] == "PH", 'tag_frequency'] = 60
    df_lines.loc[df_lines['dc'] == True, 'tag_frequency'] = 0
    df_lines.loc[df_lines['dc'] == True, 'underground'] = True
    df_lines['geometry'] = [LineString(xy) for xy in zip(df_lines['bus0'],df_lines['bus1'])]
    df_lines['bus0'] = None
    df_lines['bus1'] = None
    
    gdf_lines = gpd.GeoDataFrame(df_lines, crs="EPSG:4326")#.set_index('line_id')

    d1_buses = {'bus_id' : [bus0_sign + i for i in df.index],
                'symbol' : ["substation" for i in df.index],
                'tag_substation' : ["transmission" for i in df.index],
                'voltage' : df.Voltage * 1000,
                'lon' : df.x1,
                'lat' : df.y1,
                'dc' : df.DC,
                'under_construction' : [False for i in df.index],
                'station_id' : [None for i in df.index],
                'country' : df.Country,
                'geometry' : [Point(xy) for xy in zip(df.x1,df.y1)],
    }
    
    d2_buses = {'bus_id' : [bus1_sign + i for i in df.index],
                'symbol' : ["substation" for i in df.index],
                'tag_substation' : ["transmission" for i in df.index],
                'voltage' : df.Voltage * 1000,
                'lon' : df.x2,
                'lat' : df.y2,
                'dc' : df.DC,
                'under_construction' : [False for i in df.index],
                'station_id' : [None for i in df.index],
                'country' : df.Country,
                'geometry' : [Point(xy) for xy in zip(df.x2,df.y2)],
    }
    
    df_buses = pd.concat([pd.DataFrame(data = d1_buses),pd.DataFrame(data = d2_buses)])
    gdf_buses = gpd.GeoDataFrame(df_buses, crs="EPSG:4326")#.set_index('bus_id')

    return gdf_lines, gdf_buses

In [76]:
df = pd.read_excel('custom_int_net_v2.xlsx').drop(columns={"Num"})

int_lines, int_buses = create_custom_network(df, 0, 100000, 200000)

In [77]:
df = pd.read_excel('custom_ext_net_irena_v2.xlsx').drop(columns={"Num"})

ext_lines, ext_buses = create_custom_network(df, 1, 300000, 400000)

criteria = ext_buses.bus_id.astype("str").map(lambda x: x.startswith("4"))
ext_buses.loc[criteria, "country"] = df.loc[:,"Country_1"].tolist()

In [78]:
fn_lines = "../../../pypsa-earth/resources/" + RDIR + "osm/clean/custom_int_lines.geojson"
fn_buses = "../../../pypsa-earth/resources/" + RDIR + "osm/clean/custom_int_buses.geojson"

save_to_geojson(int_lines, fn_lines)
save_to_geojson(int_buses, fn_buses)

fn_lines = "../../../pypsa-earth/resources/" + RDIR + "osm/clean/custom_ext_lines.geojson"
fn_buses = "../../../pypsa-earth/resources/" + RDIR + "osm/clean/custom_ext_buses.geojson"

save_to_geojson(ext_lines, fn_lines)
save_to_geojson(ext_buses, fn_buses)

In [79]:
df = pd.read_excel('removed_osm_net_v2.xlsx').drop(columns={"Num"})
removed_line_list = df.id.tolist()
lines_filtered = lines.query('line_id not in @removed_line_list')

removed_buses_list = lines.query('line_id in @removed_line_list').geometry.boundary.explode(index_parts=False)
removed_buses = pd.DataFrame(removed_buses_list).rename(columns={0: "point"})
removed_buses['country'] = buses.loc[removed_buses.index,'country']

buses_filtered = buses.query('geometry not in @removed_buses_list')

In [80]:
len(removed_buses)

302

In [81]:
buses

,bus_id,symbol,tag_substation,voltage,lon,lat,dc,under_construction,station_id,tag_area,country,geometry
0,0,substation,transmission,132000,96.348292,20.794356,False,False,None,NaN,MM,POINT (96.34829 20.79436)
1,1,substation,transmission,132000,96.592176,20.643520,False,False,None,NaN,MM,POINT (96.59218 20.64352)
2,2,substation,transmission,66000,96.592176,20.643520,False,False,None,NaN,MM,POINT (96.59218 20.64352)
3,5,substation,transmission,132000,94.995165,20.129134,False,False,None,NaN,MM,POINT (94.99517 20.12913)
4,6,substation,transmission,230000,96.231276,21.857566,False,False,None,NaN,MM,POINT (96.23128 21.85757)
...,...,...,...,...,...,...,...,...,...,...,...,...
39360,39564,substation,transmission,110000,105.829141,21.126511,False,False,None,0.0,VN,POINT (105.82914 21.12651)
39361,39565,substation,transmission,110000,105.865261,20.881086,False,False,None,0.0,VN,POINT (105.86526 20.88109)
39362,39566,substation,transmission,110000,105.916028,20.759330,False,False,None,0.0,VN,POINT (105.91603 20.75933)
39363,39567,substation,transmission,110000,105.905275,20.776214,False,False,None,0.0,VN,POINT (105.90528 20.77621)


In [82]:
buses_filtered

,bus_id,symbol,tag_substation,voltage,lon,lat,dc,under_construction,station_id,tag_area,country,geometry
0,0,substation,transmission,132000,96.348292,20.794356,False,False,None,NaN,MM,POINT (96.34829 20.79436)
1,1,substation,transmission,132000,96.592176,20.643520,False,False,None,NaN,MM,POINT (96.59218 20.64352)
2,2,substation,transmission,66000,96.592176,20.643520,False,False,None,NaN,MM,POINT (96.59218 20.64352)
3,5,substation,transmission,132000,94.995165,20.129134,False,False,None,NaN,MM,POINT (94.99517 20.12913)
4,6,substation,transmission,230000,96.231276,21.857566,False,False,None,NaN,MM,POINT (96.23128 21.85757)
...,...,...,...,...,...,...,...,...,...,...,...,...
39360,39564,substation,transmission,110000,105.829141,21.126511,False,False,None,0.0,VN,POINT (105.82914 21.12651)
39361,39565,substation,transmission,110000,105.865261,20.881086,False,False,None,0.0,VN,POINT (105.86526 20.88109)
39362,39566,substation,transmission,110000,105.916028,20.759330,False,False,None,0.0,VN,POINT (105.91603 20.75933)
39363,39567,substation,transmission,110000,105.905275,20.776214,False,False,None,0.0,VN,POINT (105.90528 20.77621)


In [83]:
len(buses) - len(buses_filtered)

306

In [84]:
#from shapely.ops import nearest_points
#import math

#for index, row in removed_buses.iterrows():
#    country_filter = row.country
#    point_filter = row.point
#    xmin = math.floor(point_filter.x*10)/10
#    ymin = math.floor(point_filter.y*10)/10
#    xmax = math.ceil(point_filter.x*10)/10
#    ymax = math.ceil(point_filter.y*10)/10
#    c_buses = buses.query('lon > @xmin & lon < @xmax & lat > @ymin & lat < @ymax')

#    multipoint = c_buses.geometry.unary_union
#    queried_geom = nearest_points(point_filter, multipoint)[0]
#    #c2_buses = buses.query('geometry == @queried_geom')
#    print(queried_geom)

In [85]:
lines_merged = pd.concat([lines_filtered, int_lines, ext_lines]).reset_index().drop(columns=["index"])
buses_merged = pd.concat([buses_filtered, int_buses, ext_buses]).reset_index().drop(columns=["index"])

In [86]:

fn_lines = "../../../pypsa-earth/resources/" + RDIR + "osm/clean/all_clean_lines.geojson"
fn_buses = "../../../pypsa-earth/resources/" + RDIR + "osm/clean/all_clean_substations.geojson"

save_to_geojson(lines_merged, fn_lines)
save_to_geojson(buses_merged, fn_buses)